# Code that might be useful later

## Sklearn trick for clustering strings

In [2]:
class SklearnStringTrick:
    """
    String clustering trick shown at:
        https://scikit-learn.org/0.16/faq.html#how-do-i-deal-with-string-data-or-trees-graphs

    Class structure allows for method reuse over distinct data

    """

    def __init__(self, dataset=[]):
        self.dataset = dataset

    def lev_dist_sklearn_urls(self, x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        return levenshtein(self.dataset[i], self.dataset[j])

In [3]:
string_trick = SklearnStringTrick()

## Pattern for filtering IPv6 addresses

## My attempt for regex splitting domain and subdomains

## Alternative library for tld matching